In [122]:
%%time
# General geospatial libs
import gdal # geospatial functions
from xml.etree import ElementTree # xml
from lxml import etree # xml
gdal.UseExceptions()

Wall time: 0 ns


In [123]:
# Reading and working with rasters
# glymur for jp2.., return glymur.Jp2k(filename)[:]
import glymur # reading raster
import rasterio # reading jp2 (built on top of gdal)
import numpy as np # representing the raster data
import matplotlib.pyplot as plt # ploting images

In [124]:
# Unused libraries
# import cupy as cp #  GPU ACCELERATION FOR NUMPY
# import pandas as pd
# from io import BytesIO

### Utility functions

In [357]:
def print_meta_data(data, arguments):
    for arg in arguments:
        print(arg, ":", data[arg])

# amax = 2^16 amin=0
def linear_stretch(band, limits):
    low, high = limits
    new_band = (65535 * band) / (high - low)
    print(new_band)
    return new_band

def normalize_to_8(band, percentile=0):
#     min_d, max_d = percentile
#     band = ((band - min_d)  * 255) / (max_d - min_d) 
    # band *= 255.0/image.max() would eliminate temporary array but we need int
    # band = (band * 255) // image.max()# division on each element is slow but I don't have any better solution so far
    return band.astype('uint8')

In [358]:
linear_stretch(17510, (0,17510))

65535.0


65535.0

# Load data

In [359]:
root = "../data/"

In [360]:
dataset_1 = root + "S2A_MSIL2A_20200912T100031_N0214_R122_T33UXR_20200912T114911.SAFE/"

# Explore data

## XML data

In [128]:
MTD_MSIL2A = dataset_1 + "MTD_MSIL2A.xml"

### XML DATA WITH GDAL

In [129]:
MTD_MSIL2A_data = gdal.Open(MTD_MSIL2A)
MTD_MSIL2A_data.GetMetadata() # meta data option

{'AOT_QUANTIFICATION_VALUE': '1000.0',
 'AOT_QUANTIFICATION_VALUE_UNIT': 'none',
 'AOT_RETRIEVAL_ACCURACY': '0.0',
 'BOA_QUANTIFICATION_VALUE': '10000',
 'BOA_QUANTIFICATION_VALUE_UNIT': 'none',
 'CLOUD_COVERAGE_ASSESSMENT': '11.340936',
 'CLOUD_SHADOW_PERCENTAGE': '7.01378',
 'DARK_FEATURES_PERCENTAGE': '1.060481',
 'DATATAKE_1_DATATAKE_SENSING_START': '2020-09-12T10:00:31.024Z',
 'DATATAKE_1_DATATAKE_TYPE': 'INS-NOBS',
 'DATATAKE_1_ID': 'GS2A_20200912T100031_027289_N02.14',
 'DATATAKE_1_SENSING_ORBIT_DIRECTION': 'DESCENDING',
 'DATATAKE_1_SENSING_ORBIT_NUMBER': '122',
 'DATATAKE_1_SPACECRAFT_NAME': 'Sentinel-2A',
 'DEGRADED_ANC_DATA_PERCENTAGE': '0.0',
 'DEGRADED_MSI_DATA_PERCENTAGE': '0',
 'FOOTPRINT': 'POLYGON((16.411349506049252 50.543739174288454, 17.95938607628957 50.5146700159563, 17.899359688454204 49.52841829443089, 16.382684363173013 49.55649445196309, 16.411349506049252 50.543739174288454))',
 'FORMAT_CORRECTNESS': 'PASSED',
 'GENERAL_QUALITY': 'PASSED',
 'GENERATION_TIME':

In [130]:
datasets = MTD_MSIL2A_data.GetMetadata('SUBDATASETS')
print("Datasets\n")
for i in range(4):
    print(datasets['SUBDATASET_%d_DESC' % (i+1)])
    print("Files:", end=" ")
    print(datasets['SUBDATASET_%d_NAME' % (i+1)] + "\n")

Datasets

Bands B2, B3, B4, B8 with 10m resolution, UTM 33N
Files: SENTINEL2_L2A:../data/S2A_MSIL2A_20200912T100031_N0214_R122_T33UXR_20200912T114911.SAFE/MTD_MSIL2A.xml:10m:EPSG_32633

Bands B5, B6, B7, B8A, B11, B12 with 20m resolution, UTM 33N
Files: SENTINEL2_L2A:../data/S2A_MSIL2A_20200912T100031_N0214_R122_T33UXR_20200912T114911.SAFE/MTD_MSIL2A.xml:20m:EPSG_32633

Bands B1, B9 with 60m resolution, UTM 33N
Files: SENTINEL2_L2A:../data/S2A_MSIL2A_20200912T100031_N0214_R122_T33UXR_20200912T114911.SAFE/MTD_MSIL2A.xml:60m:EPSG_32633

True color image, UTM 33N
Files: SENTINEL2_L2A:../data/S2A_MSIL2A_20200912T100031_N0214_R122_T33UXR_20200912T114911.SAFE/MTD_MSIL2A.xml:TCI:EPSG_32633



In [131]:
# GDAL INFO way
meta_data = gdal.Info(MTD_MSIL2A, format='json')
print(gdal.Info(MTD_MSIL2A))

Driver: SENTINEL2/Sentinel 2
Files: ../data/S2A_MSIL2A_20200912T100031_N0214_R122_T33UXR_20200912T114911.SAFE/MTD_MSIL2A.xml
Size is 512, 512
Metadata:
  AOT_QUANTIFICATION_VALUE=1000.0
  AOT_QUANTIFICATION_VALUE_UNIT=none
  AOT_RETRIEVAL_ACCURACY=0.0
  BOA_QUANTIFICATION_VALUE=10000
  BOA_QUANTIFICATION_VALUE_UNIT=none
  CLOUD_COVERAGE_ASSESSMENT=11.340936
  CLOUD_SHADOW_PERCENTAGE=7.01378
  DARK_FEATURES_PERCENTAGE=1.060481
  DATATAKE_1_DATATAKE_SENSING_START=2020-09-12T10:00:31.024Z
  DATATAKE_1_DATATAKE_TYPE=INS-NOBS
  DATATAKE_1_ID=GS2A_20200912T100031_027289_N02.14
  DATATAKE_1_SENSING_ORBIT_DIRECTION=DESCENDING
  DATATAKE_1_SENSING_ORBIT_NUMBER=122
  DATATAKE_1_SPACECRAFT_NAME=Sentinel-2A
  DEGRADED_ANC_DATA_PERCENTAGE=0.0
  DEGRADED_MSI_DATA_PERCENTAGE=0
  FOOTPRINT=POLYGON((16.411349506049252 50.543739174288454, 17.95938607628957 50.5146700159563, 17.899359688454204 49.52841829443089, 16.382684363173013 49.55649445196309, 16.411349506049252 50.543739174288454))
  FORMAT_CORREC

#### Basic python built in xml parser

Looking up images

In [ ]:
tree = ElementTree.parse(MTD_MSIL2A)
root = tree.getroot()

In [213]:
def find_rec(node, element):
    item = node.findall(element)
    for child in node:
        if len(item) > 0:
            return item
        item = find_rec(child, element)
    return item

In [225]:
images = []
for image in find_rec(root, 'Granule')[0]:
    images.append(image.text)
images_spatial_20 = images[7:20] # get images with spatial resolution of 20m

### Raster data

In [226]:
raster_data = dataset_1 + 'GRANULE/L2A_T33UXR_A027289_20200912T100044/IMG_DATA/'

In [227]:
string = '../data/S2A_MSIL2A_20200912T100031_N0214_R122_T33UXR_20200912T114911.SAFE/GRANULE/L2A_T33UXR_A027289_20200912T100044/IMG_DATA/R10m/T33UXR_20200912T100031_B02_10m.jp2'

#### Glymur

##### RGB

In [230]:
print("we are working with these images: ")
images_spatial_20 # (color,band,index in array) -> we want red-B04 -> 2, green->B03->1, blue-B02-0

we are working with these images: 


['GRANULE/L2A_T33UXR_A027289_20200912T100044/IMG_DATA/R20m/T33UXR_20200912T100031_B02_20m',
 'GRANULE/L2A_T33UXR_A027289_20200912T100044/IMG_DATA/R20m/T33UXR_20200912T100031_B03_20m',
 'GRANULE/L2A_T33UXR_A027289_20200912T100044/IMG_DATA/R20m/T33UXR_20200912T100031_B04_20m',
 'GRANULE/L2A_T33UXR_A027289_20200912T100044/IMG_DATA/R20m/T33UXR_20200912T100031_B05_20m',
 'GRANULE/L2A_T33UXR_A027289_20200912T100044/IMG_DATA/R20m/T33UXR_20200912T100031_B06_20m',
 'GRANULE/L2A_T33UXR_A027289_20200912T100044/IMG_DATA/R20m/T33UXR_20200912T100031_B07_20m',
 'GRANULE/L2A_T33UXR_A027289_20200912T100044/IMG_DATA/R20m/T33UXR_20200912T100031_B8A_20m',
 'GRANULE/L2A_T33UXR_A027289_20200912T100044/IMG_DATA/R20m/T33UXR_20200912T100031_B11_20m',
 'GRANULE/L2A_T33UXR_A027289_20200912T100044/IMG_DATA/R20m/T33UXR_20200912T100031_B12_20m',
 'GRANULE/L2A_T33UXR_A027289_20200912T100044/IMG_DATA/R20m/T33UXR_20200912T100031_TCI_20m',
 'GRANULE/L2A_T33UXR_A027289_20200912T100044/IMG_DATA/R20m/T33UXR_20200912T10003

In [279]:
glym_red = dataset_1 + images_spatial_20[2] + '.jp2'
glym_green = dataset_1 + images_spatial_20[1] + '.jp2'
glym_blue = dataset_1 + images_spatial_20[0] + '.jp2'

glym_red_raster = glymur.Jp2k(glym_red)
glym_green_raster = glymur.Jp2k(glym_green)
glym_blue_raster = glymur.Jp2k(glym_blue)
# print(glym_red_raster) # print meta

In [280]:
glymur.set_option('lib.num_threads', 4) # faster reading

In [344]:
%%time
red_data = glym_red_raster[:]
blue_data = glym_blue_raster[:]
green_data = glym_green_raster[:]

Wall time: 7.65 s


In [345]:
rgb = np.dstack((red_data, blue_data, green_data))

#### Rasterio

##### Creating True color image - doesnt work

In [361]:
# No need to do this if we use our aux data but I prefer do it this way
original_tci = raster_data + 'R10m/T33UXR_20200912T100031_TCI_10m.jp2'
red_band = raster_data + 'R10m/T33UXR_20200912T100031_B04_10m.jp2'
green_band = raster_data + 'R10m/T33UXR_20200912T100031_B03_10m.jp2'
blue_band = raster_data + 'R10m/T33UXR_20200912T100031_B02_10m.jp2'

# Open images (with rasterio lib-build on top of GDAL), these are 16bit grayscale images
red_image = rasterio.open(red_band)
green_image = rasterio.open(green_band)
blue_image = rasterio.open(blue_band)

assert(red_image.count == green_image.count == blue_image.count) # 1 band each
# Load raster bands
red = red_image.read(1)
green = green_image.read(1)
blue = blue_image.read(1)

In [362]:
rgb = [red, green , blue]
p = np.percentile(rgb, (2,95))
for i in range(0,3):
    rgb[i] = normalize_to_8(rgb[i], p)

In [363]:
rgb_profile = red_image.profile
rgb_profile['dtype'] = 'uint8'
rgb_profile['count'] = 3
rgb_profile['photometric'] = "RGB"

In [364]:
with rasterio.open("rgb.jp2", 'w', **rgb_profile) as dst:
    for count,band in enumerate(rgb,1):
        dst.write(band, count)

In [365]:
rgb2 = rgb_profile
rgb_profile['dtype'] = 'uint8'
red2 = normalize_to_8(red)
green2 = normalize_to_8(green)
blue2 = normalize_to_8(blue)

In [366]:
with rasterio.open("rgb3.jp2", 'w', **rgb2) as dst:
    dst.write(red2, 1)
    dst.write(green2, 2)
    dst.write(blue2, 3)

### NDVI

In [367]:
red_band = raster_data + 'R10m/T33UXR_20200912T100031_B04_10m.jp2'
nir_band = raster_data + 'R10m/T33UXR_20200912T100031_B08_10m.jp2'

In [368]:
%%time
with rasterio.open(red_band) as red:
    red = red.read()
with rasterio.open(nir_band) as nir:
    nir = nir.read()

Wall time: 10.6 s


In [369]:
%%time
ndvi1 = (nir.astype(float) - red.astype(float))
ndvi2 = (nir+red)
ndvi = np.divide(ndvi1, ndvi2, out=np.zeros_like(ndvi1), where=ndvi2!=0)
ndvi = ndvi.squeeze()

Wall time: 3.44 s


In [ ]:
plt.imshow(ndvi.squeeze(), cmap='cividis')

In [ ]:
profile = rasterio.open(red_band).profile
profile.update(dtype='uint8')
profile['count'] = 3
profile['photometric'] = "RGB"

In [ ]:
%%time
red = cp.array(np.copy(ndvi))

In [ ]:
%%time
green = np.copy(ndvi)
blue = np.copy(ndvi)

In [ ]:
vals = [-0.2, 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0 ]
redd = [0, 165, 215, 244, 253, 254, 255, 217, 166, 102, 26, 0]
greenn = [0, 0, 48, 109, 174, 224, 255, 239, 217, 189, 152, 104]
bluee = [0, 38, 39, 67, 97, 139, 191, 139, 106, 99, 80, 55]

In [ ]:
%%time
# I AM SORRY FOR THE NEXT LINES OF CODE :( .
red[red<(-0.2)] = 0
red[(red>=(-0.2))&(red<=0)] = 165
red[(red>.0)&(red<=.1)] = 215
red[(red>.1)&(red<=.2)] = 244
red[(red>.2)&(red<=.3)] = 253
red[(red>.3)&(red<=.4)] = 254
red[(red>.4)&(red<=.5)] = 255
red[(red>.5)&(red<=.6)] = 217
red[(red>.6)&(red<=.7)] = 166
red[(red>.7)&(red<=.8)] = 102
red[(red>.8)&(red<=.9)] = 26
red[(red>.9)&(red<=1)] = 0
print("Time achieved with GPU")

In [ ]:
%%time
green[green<(-0.2)] = 0
green[(green>=(-0.2))&(green<=0)] = 0
green[(green>.0)&(green<=.1)] = 48
green[(green>.1)&(green<=.2)] = 109
green[(green>.2)&(green<=.3)] = 174
green[(green>.3)&(green<=.4)] = 224
green[(green>.4)&(green<=.5)] = 255
green[(green>.5)&(green<=.6)] = 239
green[(green>.6)&(green<=.7)] = 217
green[(green>.7)&(green<=.8)] = 189
green[(green>.8)&(green<=.9)] = 152
green[(green>.9)&(green<=1)] = 104
print("Time achieved with CPU:")

In [ ]:
blue[blue<(-0.2)] = 0
blue[(blue>=(-0.2))&(blue<=0)] = 38
blue[(blue>.0)&(blue<=.1)] = 39
blue[(blue>.1)&(blue<=.2)] = 67
blue[(blue>.2)&(blue<=.3)] = 97
blue[(blue>.3)&(blue<=.4)] = 139
blue[(blue>.4)&(blue<=.5)] = 191
blue[(blue>.5)&(blue<=.6)] = 139
blue[(blue>.6)&(blue<=.7)] = 106
blue[(blue>.7)&(blue<=.8)] = 99
blue[(blue>.8)&(blue<=.9)] = 80
blue[(blue>.9)&(blue<=1)] = 55

In [ ]:
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 0
print(pinned_mempool.n_free_blocks())  

red_cpu = cp.asnumpy(red).astype('uint8')
red = None
mempool.free_all_blocks()
pinned_mempool.free_all_blocks()
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 0
print(pinned_mempool.n_free_blocks())  

green = green.astype('uint8')
blue = blue.astype('uint8')

In [ ]:
plt.imshow(red_cpu, cmap='Reds')

In [ ]:
plt.imshow(blue, cmap='Blues')

In [ ]:
with rasterio.open('ndvi.jp2', 'w', **profile) as dst:
    dst.write(red, 1)
    dst.write(green, 2)
    dst.write(blue, 3)

### NDVI CLOUD MASK

In [ ]:
ndvi = ndvi.squeeze()

In [ ]:
mask = np.where(ndvi<=.1, 0, 1).astype('uint8')

In [ ]:
cloud_profile = rasterio.open(red_band).profile
cloud_profile['dtype'] = 'uint8'

In [ ]:
with rasterio.open('ndvi_mask.jp2', 'w', **profile) as dst:
    dst.write(mask, 1)

In [ ]:
plt.imshow(mask, cmap='binary')